# Scraping Fundación Lealtad

## Bibliotecas

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Lista de ligas

In [5]:
# URL of the page to scrape
url = "https://www.fundacionlealtad.org/ong/"

# Set headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Send a GET request to the page with headers
response = requests.get(url, headers=headers)
response.raise_for_status()  # Check if the request was successful

# Parse the HTML content of the page
soup = BeautifulSoup(response.text, 'html.parser')

# Find all <a> tags with the specific class
a_tags = soup.find_all('a', class_='fl-list__item-link')

# Extract the href attributes and form the full URLs
links = ["https://www.fundacionlealtad.org" + a['href'] for a in a_tags]

# Print the list of links
for link in links:
    print(link)

# If you want to store the links in a list
link_list = list(links)

https://www.fundacionlealtad.org/ong/a-toda-vela
https://www.fundacionlealtad.org/ong/accion-por-la-musica
https://www.fundacionlealtad.org/ong/adacen-asociacion-de-dano-cerebral-de-navarra
https://www.fundacionlealtad.org/ong/adcor-adultos-discapacitados-de-a-coruna
https://www.fundacionlealtad.org/ong/adela-asociacion-espanola-de-esclerosis-lateral-amiotrofica
https://www.fundacionlealtad.org/ong/adem-collado-villalba
https://www.fundacionlealtad.org/ong/ademna-asociacion-de-esclerosis-multiple-de-navarra
https://www.fundacionlealtad.org/ong/ademto-asociacion-de-esclerosis-multiple-de-toledo
https://www.fundacionlealtad.org/ong/adepsi
https://www.fundacionlealtad.org/ong/adfypse
https://www.fundacionlealtad.org/ong/adipa-asociacion-de-discapacitados-psiquicos-de-antequera-y-de-su-comarca
https://www.fundacionlealtad.org/ong/adisli-atencion-de-personas-con-discapacidad-intelectual-ligera-e-inteligencia-limite
https://www.fundacionlealtad.org/ong/asociacion-de-familiares-y-afectados-po

In [6]:
len(link_list)

282

Pruebo con una liga

In [25]:
# URL of the first NGO page to scrape
url = "https://www.fundacionlealtad.org/ong/a-toda-vela/"

# Set headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Send a GET request to the page with headers
response = requests.get(url, headers=headers)
response.raise_for_status()  # Check if the request was successful

# Parse the HTML content of the page
soup = BeautifulSoup(response.text, 'html.parser')

# Extract data
nombre = soup.find('h1', class_='ngo-card__name').text.strip()
mision = soup.find('section', class_='ngo-card__section--mission').find('p').text.strip()
origen_section = soup.find('section', class_='ngo-card__section--origin')
origen = origen_section.find('p').text.strip()
anio_constitucion = origen_section.find_all('span', class_='ngo-card__origin-data-value')[0].text.strip()
personalidad_juridica = origen_section.find_all('span', class_='ngo-card__origin-data-value')[1].text.strip()
actividades = soup.find('section', class_='ngo-card__section--activity').find('p').text.strip()
beneficiarios = soup.find('td', class_='ngo-card__activity-table-data').text.strip()
socios = soup.find_all('td', class_='ngo-card__activity-table-data')[1].text.strip()
voluntarios = soup.find_all('td', class_='ngo-card__activity-table-data')[2].text.strip()
empleados = soup.find_all('td', class_='ngo-card__activity-table-data')[3].text.strip()

# Extracting Ingresos and Gastos from both summary and charts
chart_summary = soup.find('div', class_='ngo-card__chart-summary-container').find_all('li')
print(chart_summary)
ingresos_summary = chart_summary[1].text.split()[1]
gastos_summary = chart_summary[2].text.split()[1]
print(ingresos_summary,gastos_summary)
chart_titles = soup.find_all('span', class_='js__donut-title')
print(chart_titles)
# ingresos_chart = chart_titles[0].text.split()[1]
# gastos_chart = chart_titles[1].text.split()[1]

presidente = soup.find('div', class_='ngo-card__gobernance').find_all('span', class_='ngo-card__gobernance-value')[0].text.strip()
director = soup.find('div', class_='ngo-card__gobernance').find_all('span', class_='ngo-card__gobernance-value')[1].text.strip()

# Create a table with the extracted data
data = {
    "Nombre": [nombre],
    "Misión": [mision],
    "Origen": [origen],
    "Año de Constitución": [anio_constitucion],
    "Personalidad Jurídica": [personalidad_juridica],
    "Actividades": [actividades],
    "Beneficiarios": [beneficiarios],
    "Socios": [socios],
    "Voluntarios": [voluntarios],
    "Empleados": [empleados],
    # "Ingresos": [ingresos],
    # "Gastos": [gastos],
    "Presidente": [presidente],
    "Director": [director]
}

# Convert the data to a DataFrame
df = pd.DataFrame(data)

# Print the DataFrame
df

[<li>(2022)</li>, <li>Ingresos 702.668 €</li>, <li>Gastos 625.685 €</li>]
702.668 625.685
[<span class="js__donut-title"></span>, <span class="js__donut-title"></span>]


,Nombre,Misión,Origen,Año de Constitución,Personalidad Jurídica,Actividades,Beneficiarios,Socios,Voluntarios,Empleados,Presidente,Director
0,A Toda Vela,"""Contribuir a la mejora de la calidad de vida ...",Nace en Almería por el impulso de cinco mujere...,1997,Asociación,A Toda Vela atiende a personas con discapacida...,192,613,60,24,María Dolores Manzanares Martínez,Rebeca Tur Barja


Scrapeamos todo

In [20]:
def scrape_ngo_data(urls):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    all_data = []

    for url in urls:
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')

        try:
            nombre = soup.find('h1', class_='ngo-card__name').text.strip()
        except AttributeError:
            nombre = None

        try:
            mision = soup.find('section', class_='ngo-card__section--mission').find('p').text.strip()
        except AttributeError:
            mision = None

        try:
            origen_section = soup.find('section', class_='ngo-card__section--origin')
            origen = origen_section.find('p').text.strip()
            anio_constitucion = origen_section.find_all('span', class_='ngo-card__origin-data-value')[0].text.strip()
            personalidad_juridica = origen_section.find_all('span', class_='ngo-card__origin-data-value')[1].text.strip()
        except (AttributeError, IndexError):
            origen = anio_constitucion = personalidad_juridica = None

        try:
            actividades = soup.find('section', class_='ngo-card__section--activity').find('p').text.strip()
        except AttributeError:
            actividades = None

        try:
            beneficiarios = soup.find('td', class_='ngo-card__activity-table-data').text.strip()
            socios = soup.find_all('td', class_='ngo-card__activity-table-data')[1].text.strip()
            voluntarios = soup.find_all('td', class_='ngo-card__activity-table-data')[2].text.strip()
            empleados = soup.find_all('td', class_='ngo-card__activity-table-data')[3].text.strip()
        except (AttributeError, IndexError):
            beneficiarios = socios = voluntarios = empleados = None

        try:
            chart_summary = soup.find('div', class_='ngo-card__chart-summary-container').find_all('li')
            ingresos = chart_summary[1].text.split()[1]
            gastos = chart_summary[2].text.split()[1]
        except (AttributeError, IndexError):
            ingresos = gastos = None

        try:
            presidente = soup.find('div', class_='ngo-card__gobernance').find_all('span', class_='ngo-card__gobernance-value')[0].text.strip()
            director = soup.find('div', class_='ngo-card__gobernance').find_all('span', class_='ngo-card__gobernance-value')[1].text.strip()
        except (AttributeError, IndexError):
            presidente = director = None

        data = {
            "Nombre": nombre,
            "Misión": mision,
            "Origen": origen,
            "Año de Constitución": anio_constitucion,
            "Personalidad Jurídica": personalidad_juridica,
            "Actividades": actividades,
            "Beneficiarios": beneficiarios,
            "Socios": socios,
            "Voluntarios": voluntarios,
            "Empleados": empleados,
            "Ingresos": ingresos,
            "Gastos": gastos,
            "Presidente": presidente,
            "Director": director
        }

        all_data.append(data)
        print(url)

    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(all_data)

    return df

# # Example usage with a list of URLs
# urls = [
#     "https://www.fundacionlealtad.org/ong/a-toda-vela/",
#     # Add more URLs as needed
# ]

# Scrape data and print the DataFrame
df = scrape_ngo_data(link_list)
df

https://www.fundacionlealtad.org/ong/a-toda-vela
https://www.fundacionlealtad.org/ong/accion-por-la-musica
https://www.fundacionlealtad.org/ong/adacen-asociacion-de-dano-cerebral-de-navarra
https://www.fundacionlealtad.org/ong/adcor-adultos-discapacitados-de-a-coruna
https://www.fundacionlealtad.org/ong/adela-asociacion-espanola-de-esclerosis-lateral-amiotrofica
https://www.fundacionlealtad.org/ong/adem-collado-villalba
https://www.fundacionlealtad.org/ong/ademna-asociacion-de-esclerosis-multiple-de-navarra
https://www.fundacionlealtad.org/ong/ademto-asociacion-de-esclerosis-multiple-de-toledo
https://www.fundacionlealtad.org/ong/adepsi
https://www.fundacionlealtad.org/ong/adfypse
https://www.fundacionlealtad.org/ong/adipa-asociacion-de-discapacitados-psiquicos-de-antequera-y-de-su-comarca
https://www.fundacionlealtad.org/ong/adisli-atencion-de-personas-con-discapacidad-intelectual-ligera-e-inteligencia-limite
https://www.fundacionlealtad.org/ong/asociacion-de-familiares-y-afectados-po

,Nombre,Misión,Origen,Año de Constitución,Personalidad Jurídica,Actividades,Beneficiarios,Socios,Voluntarios,Empleados,Ingresos,Gastos,Presidente,Director
0,A Toda Vela,"""Contribuir a la mejora de la calidad de vida ...",Nace en Almería por el impulso de cinco mujere...,1997,Asociación,A Toda Vela atiende a personas con discapacida...,192,613,60,24,702.668,625.685,María Dolores Manzanares Martínez,Rebeca Tur Barja
1,Acción Por la Música,"""Lograr la inclusión de niños y niñas, jóvenes...",Acción Por la Música se creó en el año 2013 de...,2013,Fundación,La actividad principal de Acción Por la Música...,440,3,12,20,299.518,283.486,María Guerrero Sanz,Cristina Sitz Gento
2,ADACEN,"""Trabajar con las personas con DCA (Daño Cereb...",ADACEN fue constituida en 1994 por un grupo de...,1994,Asociación,ADACEN desarrolla las siguientes actividades: ...,314,926,57,49,None,None,José Luis Herrera Zubeldía,Andrés Ilundáin Esquíroz
3,ADCOR,"""La Fundación ADCOR es una organización sin án...","En 1996, para ofrecer una alternativa para el ...",1996,Fundación,Fundación ADCOR atiende a personas mayores en ...,152,12,11,47,2.307.119,2.336.792,Eudoxia María Neira Fernández,Margarita Garrido Fernández
4,adELA,"""Mejorar la calidad de vida de las personas af...","En 1990, el neurólogo Jesús Mora con un grup...",1990,Asociación,adELA realiza actividades para la atención de ...,2.042,1.120,38,3,782.835,706.624,María del Carmen Martínez Sarmiento,Rosa Mª Sanz Palomero
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,Sonrisas de Bombay,"""Transformar la realidad de personas vulnerabl...",La Asociación Sonrisas de Bombay (Mumbai Smile...,2011,Fundación,Sonrisas de Bombay lleva a cabo las siguientes...,4.045,2.724,58,8,None,None,Felicitat Barragán Candalia,Jaume Sanllorente Trepat
278,STOP Accidentes,"""Concienciar a la opinión pública, incluyendo ...",STOP Accidentes es una organización sin ánimo ...,2000,Asociación,STOP Accidentes trabaja por la seguridad vial ...,202.578,477,21,12,501.597,553.216,Ana Novella Reig,No cuenta con esta figura
279,Upacesur Atiende,"""Proporcionar una atención de calidad a las pe...",En 1978 nace en Cádiz la asociación Unión Pará...,2004,Fundación,Upacesur Atiende trabaja con personas con pará...,699,16,5,104,None,None,Germán Fernández Pajares,Rafael Márquez Romero
280,World Vision España,"""Trabajar con los niños y niñas más vulnerable...","World Vision tiene su origen en 1947, cuando e...",2001,Fundación,World Vision España financia programas a largo...,304.475,11.639,4,22,7.266.018,7.207.940,Ramón Taix Rodriguez-Moñino,Javier Ruiz Gaitan


Guardar a csv

In [21]:
# Save DataFrame to a CSV file
df.to_csv('./data/fundacion_lealtad.csv', index=False)